First, get gutenberg text from 'Plain Text UTF-8' and then save the text to TXT files. let's cut off garbage lines later

In [ ]:
import pandas as pd
file_path = '/content/drive/MyDrive/hdw_annotation/corpus_metadata_7_1.xlsx'
df = pd.read_excel(file_path)

In [ ]:
def modify_url(url):
  if pd.isna(url) or not url.strip():
    return None
  return url.replace('-images.html', '.txt')

In [ ]:
df['new_gutenberg_url'] = df['gutenberg_url'].apply(modify_url)

In [ ]:
df['new_gutenberg_url'][0]

'https://www.gutenberg.org/cache/epub/36481/pg36481.txt'

In [ ]:
df.to_csv('/content/drive/MyDrive/hdw_annotation/new_gutenberg_url.csv',encoding='utf-8', index=False)

In [ ]:
#this code takes time
import os
import requests
from bs4 import BeautifulSoup

output_dir = '/content/drive/MyDrive/hdw_annotation/raw_files'
os.makedirs(output_dir, exist_ok=True)

def fetch_and_save_text(row):
    url = row['new_gutenberg_url']
    filename = row['filename']

    if pd.isna(url) or not url.strip():
        return

    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            body_text = soup.get_text()
            #save the body of each text as follows
            file_path = os.path.join(output_dir, f"{filename}.txt")
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(body_text)
        else:
            print(f"Failed to fetch {url}: Status code {response.status_code}")
    except Exception as e:
        print(f"Error fetching {url}: {e}")


df.apply(fetch_and_save_text, axis=1)

Failed to fetch https://www.gutenberg.org/cache/epub/35786pg-35786images.html: Status code 404
Failed to fetch https://www.gutenberg.org/cache/epub/2505/pgv.txt: Status code 404


0      None
1      None
2      None
3      None
4      None
       ... 
647    None
648    None
649    None
650    None
651    None
Length: 652, dtype: object

I added the two texts whose urls are not available manually.

Regarding the filenames of the generated texts, there are redundant '.txt' (like 'ainsworth-bourbon-1866.txt.txt'). let's cut off the last four digits (.txt)

In [ ]:
for filename in os.listdir('/content/drive/MyDrive/hdw_annotation/raw_files'):
    if filename.endswith(".txt.txt"):
        new_filename = filename[:-4]
        old_file_path = os.path.join(directory, filename)
        new_file_path = os.path.join(directory, new_filename)
        os.rename(old_file_path, new_file_path)

Using the code in the following link, let's remove

---

headers and footers
#https://github.com/kiasar/gutenberg_cleaner/blob/master/_cleaning_options/strip_headers.py

In [ ]:
"""Module to remove the noise from Project Gutenberg texts."""

from __future__ import absolute_import, unicode_literals
from builtins import str
import os

TEXT_START_MARKERS = frozenset((
    "*END*THE SMALL PRINT",
    "*** START OF THE PROJECT GUTENBERG",
    "*** START OF THIS PROJECT GUTENBERG",
    "This etext was prepared by",
    "E-text prepared by",
    "Produced by",
    "Distributed Proofreading Team",
    "Proofreading Team at http://www.pgdp.net",
    "http://gallica.bnf.fr)",
    "      http://archive.org/details/",
    "http://www.pgdp.net",
    "by The Internet Archive)",
    "by The Internet Archive/Canadian Libraries",
    "by The Internet Archive/American Libraries",
    "public domain material from the Internet Archive",
    "Internet Archive)",
    "Internet Archive/Canadian Libraries",
    "Internet Archive/American Libraries",
    "material from the Google Print project",
    "*END THE SMALL PRINT",
    "***START OF THE PROJECT GUTENBERG",
    "This etext was produced by",
    "*** START OF THE COPYRIGHTED",
    "The Project Gutenberg",
    "http://gutenberg.spiegel.de/ erreichbar.",
    "Project Runeberg publishes",
    "Beginning of this Project Gutenberg",
    "Project Gutenberg Online Distributed",
    "Gutenberg Online Distributed",
    "the Project Gutenberg Online Distributed",
    "Project Gutenberg TEI",
    "This eBook was prepared by",
    "http://gutenberg2000.de erreichbar.",
    "This Etext was prepared by",
    "This Project Gutenberg Etext was prepared by",
    "Gutenberg Distributed Proofreaders",
    "Project Gutenberg Distributed Proofreaders",
    "the Project Gutenberg Online Distributed Proofreading Team",
    "**The Project Gutenberg",
    "*SMALL PRINT!",
    "More information about this book is at the top of this file.",
    "tells you about restrictions in how the file may be used.",
    "l'authorization à les utilizer pour preparer ce texte.",
    "of the etext through OCR.",
    "*****These eBooks Were Prepared By Thousands of Volunteers!*****",
    "We need your donations more than ever!",
    " *** START OF THIS PROJECT GUTENBERG",
    "****     SMALL PRINT!",
    '["Small Print" V.',
    '      (http://www.ibiblio.org/gutenberg/',
    'and the Project Gutenberg Online Distributed Proofreading Team',
    'Mary Meehan, and the Project Gutenberg Online Distributed Proofreading',
    '                this Project Gutenberg edition.',
))

TEXT_END_MARKERS = frozenset((
    "*** END OF THE PROJECT GUTENBERG",
    "*** END OF THIS PROJECT GUTENBERG",
    "***END OF THE PROJECT GUTENBERG",
    "End of the Project Gutenberg",
    "End of The Project Gutenberg",
    "Ende dieses Project Gutenberg",
    "by Project Gutenberg",
    "End of Project Gutenberg",
    "End of this Project Gutenberg",
    "Ende dieses Projekt Gutenberg",
    "        ***END OF THE PROJECT GUTENBERG",
    "*** END OF THE COPYRIGHTED",
    "End of this is COPYRIGHTED",
    "Ende dieses Etextes ",
    "Ende dieses Project Gutenber",
    "Ende diese Project Gutenberg",
    "**This is a COPYRIGHTED Project Gutenberg Etext, Details Above**",
    "Fin de Project Gutenberg",
    "The Project Gutenberg Etext of ",
    "Ce document fut presente en lecture",
    "Ce document fut présenté en lecture",
    "More information about this book is at the top of this file.",
    "We need your donations more than ever!",
    "END OF PROJECT GUTENBERG",
    " End of the Project Gutenberg",
    " *** END OF THIS PROJECT GUTENBERG",
))

LEGALESE_START_MARKERS = frozenset(("<<THIS ELECTRONIC VERSION OF",))

LEGALESE_END_MARKERS = frozenset(("SERVICE THAT CHARGES FOR DOWNLOAD",))


def _strip_headers(text):
    """Remove lines that are part of the Project Gutenberg header or footer.
    Note: The original version of the code can be found at:
    https://github.com/c-w/gutenberg/blob/master/gutenberg/cleanup/strip_headers.py
    Args:
        text (unicode): The body of the text to clean up.
    Returns:
        unicode: The text with any non-text content removed.
    """
    lines = text.splitlines()
    sep = str(os.linesep)

    out = []
    i = 0
    footer_found = False
    ignore_section = False

    for line in lines:
        reset = False

        if i <= 600:
            # Check if the header ends here
            if any(line.startswith(token) for token in TEXT_START_MARKERS):
                reset = True

            # If it's the end of the header, delete the output produced so far.
            # May be done several times, if multiple lines occur indicating the
            # end of the header
            if reset:
                out = []
                continue

        if i >= 100:
            # Check if the footer begins here
            if any(line.startswith(token) for token in TEXT_END_MARKERS):
                footer_found = True

            # If it's the beginning of the footer, stop output
            if footer_found:
                break

        if any(line.startswith(token) for token in LEGALESE_START_MARKERS):
            ignore_section = True
            continue
        elif any(line.startswith(token) for token in LEGALESE_END_MARKERS):
            ignore_section = False
            continue

        if not ignore_section:
            out.append(line.rstrip(sep))
            i += 1

    return sep.join(out)

In [ ]:
with open('/content/drive/MyDrive/hdw_annotation/raw_files/ainsworth-auriol-1850.txt', 'r', encoding='utf-8') as file:
  text = file.read()
test=_strip_headers(text)
test[:200]


'\n\n\n\n\n\n\n\n\n\n                            AURIOL\n\n                              OR\n\n                      THE ELIXIR OF LIFE\n\n                   BY W. HARRISON AINSWORTH\n\n                AUTHOR OF "THE TO'

In [ ]:
#let's check the content of test

with open('/content/drive/MyDrive/hdw_annotation/test_ainsworth_cleanedup.txt','w', encoding='utf-8') as file:
  file.write(test)

#this file still contains title and table of contents. But all headers and footers are removed

let's remove headers and footers using the code in (https://github.com/kiasar/gutenberg_cleaner/blob/master/_cleaning_options/strip_headers.py) for all files in 'raw_files'

In [ ]:
input_dir = '/content/drive/MyDrive/hdw_annotation/raw_files'
output_dir = '/content/drive/MyDrive/hdw_annotation/clean_files'


for filename in os.listdir(input_dir):
    if filename.endswith('.txt'):
        input_file_path = os.path.join(input_dir, filename)

        with open(input_file_path, 'r', encoding='utf-8') as file:
            text = file.read()

        cleaned_text = _strip_headers(text)#apply the cleanup function here

        output_file_path = os.path.join(output_dir, filename)

        with open(output_file_path, 'w', encoding='utf-8') as file:
            file.write(cleaned_text)
print('done')



done


Next, Claudia told me to cut off titles, tables, and contents.
To do this, Search if the text contains "PROLOGUE\n\n". If so, cut off everything before "Prologue\n\n".
If the text doesn't contain "PROLOGUE\n\n", cut off everything before "CHAPTER I\n\n"

However, this doesn't work due to lots of variations regarding the beginnings of chapters.

So, I skipped this process so far.

let's split the cleaned TXT files into 30 chunks based on the word count. 

In [9]:
import os
import re

input_dir = '/home/c/Downloads/clean_files'
output_dir = '/home/c/Downloads/chunked_texts'
os.makedirs(output_dir, exist_ok=True)

def split_text_into_chunks(text, num_chunks):
    words = re.findall(r'\S+\s*', text)
    chunk_size = len(words) // num_chunks #integer division, pay attention to the remainder later
    chunks = []

    for i in range(0, len(words), chunk_size):
        chunk = ''.join(words[i:i + chunk_size])
        chunks.append(chunk)

    # If there are fewer chunks than num_chunks due to rounding, merge the last few words
    if len(chunks) > num_chunks:
        chunks[-2] += chunks[-1]
        chunks = chunks[:-1]

    return chunks

num_chunks = 30

for filename in os.listdir(input_dir):
    if filename.endswith('.txt'):
        input_file_path = os.path.join(input_dir, filename)

        with open(input_file_path, 'r', encoding='utf-8') as file:
            text = file.read()

        chunks = split_text_into_chunks(text, num_chunks)

        base_filename = os.path.splitext(filename)[0]
        for i, chunk in enumerate(chunks, 1):
            output_filename = f"{base_filename}_{i}.txt"
            output_file_path = os.path.join(output_dir, output_filename)

            with open(output_file_path, 'w', encoding='utf-8') as chunk_file:
                chunk_file.write(chunk)

print('done')


done


\S+:  matches one or more (+) non-whitespace characters (\S). "\S" represents any character except whitespace (space, tab, newline, etc.).
\s*:  matches zero or more (*) whitespace characters (\s).

In [11]:
# Count the number of files
files = os.listdir('/home/c/Downloads/chunked_texts')
len([f for f in files if os.path.isfile(os.path.join('/home/c/Downloads/chunked_texts', f))])

18061

In [12]:
files = os.listdir('/home/c/Downloads/clean_files')
len([f for f in files if os.path.isfile(os.path.join('/home/c/Downloads/clean_files', f))])

602

number of files in chunked_texts look acurate

The problem regarding the chunked texts above is, sometimes dialogues at the end of a chunk is split and the part of the dialogue is saved in the next chunk. If this happens, the next chunk starts from part of dialogue, which lacks bedinning quoataion marks.

To solve this issue, I need to put the first passage before the first double new lines in each chunk into the previous chunk.


In [13]:
# put the first passage before the first double new lines in each chunk into the previous chunk.
import os
import re
from collections import defaultdict

source_folder = '/home/c/Downloads/chunked_texts'
dest_folder = '/home/c/Downloads/updated_chunked_texts'

os.makedirs(dest_folder, exist_ok=True)

text_chunks = defaultdict(dict)

# Read all chunks for each text
for filename in os.listdir(source_folder):
    match = re.match(r'(.+)_(\d+)\.txt', filename)
    if match:
        base_name, chunk_num = match.groups()
        chunk_num = int(chunk_num)
        file_path = os.path.join(source_folder, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            text_chunks[base_name][chunk_num] = content

# Extract first part before double newlines and move to previous chunk
for base_name, chunks in text_chunks.items():
    chunk_numbers = sorted(chunks.keys())
    for i in range(1, len(chunk_numbers)):
        current_chunk = chunk_numbers[i]
        prev_chunk = chunk_numbers[i-1]
        content = chunks[current_chunk]
        parts = content.split('\n\n', 1)
        if len(parts) > 1:
            first_part, rest = parts
            # Update current chunk
            chunks[current_chunk] = rest
            # Append first_part to the end of the previous chunk
            chunks[prev_chunk] = chunks[prev_chunk] + first_part

# let's Save updated chunks to new folder
for base_name, chunks in text_chunks.items():
    for chunk_num, content in chunks.items():
        new_filename = f"{base_name}_{chunk_num}.txt"
        new_file_path = os.path.join(dest_folder, new_filename)
        with open(new_file_path, 'w', encoding='utf-8') as file:
            file.write(content)

print("done")

done


In [14]:
#cut off dialogues 
import glob

def extract_text(text):
    # This pattern matches dialogues starting and ending with either single or double quotes
    pattern = r'“[^”]*”|‘[^’]*’|"[^"]*"|\'[^\']*\''  # Account for straight quotes
    return re.sub(pattern, "", text)

input_dir = '/home/c/Downloads/updated_chunked_texts'
output_dir = '/home/c/Downloads/chunked_texts_no_dialogues'
os.makedirs(output_dir, exist_ok=True)

txt_files = glob.glob(os.path.join(input_dir, '*.txt'))

for txt_file in txt_files:
    with open(txt_file, 'r', encoding='utf-8') as file:
        text_content = file.read()

    # Split the text into paragraphs based on double new lines (representing blank lines)
    paragraphs = text_content.split('\n\n')

    # Clean up each paragraph by removing single new lines
    cleaned_paragraphs = [' '.join(paragraph.splitlines()) for paragraph in paragraphs]

    processed_paragraphs = []

    #aacount for long dialogues lacking closing quotation marks in each paragraph
    quote_marks = ['"', "'", '”', '’']
    for paragraph in cleaned_paragraphs:
        stripped_paragraph = paragraph.strip()
        if any(stripped_paragraph.startswith(quote) and quote not in stripped_paragraph[1:] for quote in quote_marks):
            continue
        processed_paragraph = extract_text(stripped_paragraph)
        processed_paragraphs.append(processed_paragraph.strip())

    # Remove any empty paragraphs resulting from the dialogue removal
    non_blank_paragraphs = [paragraph for paragraph in processed_paragraphs if paragraph]
    final_text = '\n'.join(non_blank_paragraphs)

    output_file_path = os.path.join(output_dir, os.path.basename(txt_file))
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(final_text)

print("done")


done


In [15]:
#split the text into full sentence based on nltk (csv1) 

import csv
import nltk
from nltk.tokenize import sent_tokenize


nltk.download('punkt')

txt_files = glob.glob(os.path.join('/home/c/Downloads/chunked_texts_no_dialogues', '*.txt'))


with open('/home/c/Downloads/sentences_nltk.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Filename', 'Sentence'])

    for txt_file in txt_files:
        with open(txt_file, 'r', encoding='utf-8') as file:
            text_content = file.read()

        sentences = sent_tokenize(text_content)

        for sentence in sentences:
            csvwriter.writerow([os.path.basename(txt_file), sentence])

print("done")


[nltk_data] Downloading package punkt to /home/c/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


done


In [16]:
import pandas as pd 

df=pd.read_csv('/home/c/Downloads/sentences_nltk.csv')
df.head(30)

,Filename,Sentence
0,james-eagle-1854_25.txt,CHAPTER XXXIX.
1,james-eagle-1854_25.txt,The lodge of Black Eagle's sister was next in ...
2,james-eagle-1854_25.txt,"Though on a somewhat smaller scale, it was ver..."
3,james-eagle-1854_25.txt,"In a word, It was only of one story, however;..."
4,james-eagle-1854_25.txt,"The possessor of this mansion, who was held in..."
5,james-eagle-1854_25.txt,"When Otaitsa approached the door, she found it..."
6,james-eagle-1854_25.txt,"A young girl then peeped out, and, seeing the ..."
7,james-eagle-1854_25.txt,The space on that side of the large mat which ...
8,james-eagle-1854_25.txt,The feelings of Otaitsa's heart were too power...
9,james-eagle-1854_25.txt,The scene was neither of a beautiful nor of a ...


In [17]:
pd.set_option('display.max_colwidth', None)
df['Sentence'][:10]

0                                                                                                                                                                                                                                                                                                                                                                                                                                                   CHAPTER XXXIX.
1                                                                                                                                                                                                                                                                                                           The lodge of Black Eagle's sister was next in size and importance to that of the chief himself, and on it, too, some European skill had been expended.
2                                                                                                 

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2173901 entries, 0 to 2173900
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Filename  object
 1   Sentence  object
dtypes: object(2)
memory usage: 33.2+ MB


In [1]:
#split based on commas and semicolons

import pandas as pd
import re

df = pd.read_csv('/home/c/Downloads/sentences_nltk.csv')


results = []

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    filename = row['Filename']
    sentences = row['Sentence']
    split_sentences = [s.strip() for s in re.split('[,;]', sentences) if s.strip()]
    for sentence in split_sentences:
        results.append({'Filename': filename, 'Sentence': sentence})


output_df = pd.DataFrame(results)
output_df.to_csv('/home/c/Downloads/split_sentences_comma_semicolon.csv', index=False)

print('done')




done


In [2]:
df3=pd.read_csv('split_sentences_comma_semicolon.csv')
df3.head(10)



,Filename,Sentence
0,james-eagle-1854_25.txt,CHAPTER XXXIX.
1,james-eagle-1854_25.txt,The lodge of Black Eagle's sister was next in ...
2,james-eagle-1854_25.txt,and on it
3,james-eagle-1854_25.txt,too
4,james-eagle-1854_25.txt,some European skill had been expended.
5,james-eagle-1854_25.txt,Though on a somewhat smaller scale
6,james-eagle-1854_25.txt,it was very much such another building as that...
7,james-eagle-1854_25.txt,In a word
8,james-eagle-1854_25.txt,It was only of one story
9,james-eagle-1854_25.txt,however


In [3]:
df3["Sentence"][:10]

0                                       CHAPTER XXXIX.
1    The lodge of Black Eagle's sister was next in ...
2                                            and on it
3                                                  too
4               some European skill had been expended.
5                   Though on a somewhat smaller scale
6    it was very much such another building as that...
7                                            In a word
8                             It was only of one story
9                                              however
Name: Sentence, dtype: object

In [4]:

#this code will split the sentences generated by nltk based on colons and semicolons



df = pd.read_csv('/home/c/Downloads/sentences_nltk.csv')


results = []

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    filename = row['Filename']
    sentences = row['Sentence']
    split_sentences = [s.strip() for s in re.split('[:;]', sentences) if s.strip()]
    for sentence in split_sentences:
        results.append({'Filename': filename, 'Sentence': sentence})


output_df = pd.DataFrame(results)
output_df.to_csv('/home/c/Downloads/split_sentences_colon_semicolon.csv', index=False)

print('done')




done


In [5]:
df4=pd.read_csv('/home/c/Downloads/split_sentences_colon_semicolon.csv')
df4.head()


,Filename,Sentence
0,james-eagle-1854_25.txt,CHAPTER XXXIX.
1,james-eagle-1854_25.txt,The lodge of Black Eagle's sister was next in ...
2,james-eagle-1854_25.txt,"Though on a somewhat smaller scale, it was ver..."
3,james-eagle-1854_25.txt,"In a word, It was only of one story, however"
4,james-eagle-1854_25.txt,but the workmen who had erected it a good many...


In [6]:
pd.set_option('display.max_colwidth', None)
df4['Sentence'][:30]

0                                                                                                                                                                                                                                                                                                                                                                                                                                                    CHAPTER XXXIX.
1                                                                                                                                                                                                                                                                                                            The lodge of Black Eagle's sister was next in size and importance to that of the chief himself, and on it, too, some European skill had been expended.
2                                                                                               

In [2]:
#compile dialogues in a different CSV file 

import os
import re
import pandas as pd

input_folder = '/home/c/Downloads/updated_chunked_texts'
output_csv = '/home/c/Downloads/dialogues.csv'

pattern = r'“[^”]*”|‘[^’]*’|"[^"]*"|\'[^\']*\''

results = []

for filename in os.listdir(input_folder):
    if filename.endswith('.txt'):
        filepath = os.path.join(input_folder, filename)
        with open(filepath, 'r', encoding='utf-8') as file:
            content = file.read()
            dialogues = re.findall(pattern, content)
            for dialogue in dialogues:
                results.append({'Filename': filename, 'Sentence': dialogue})

df = pd.DataFrame(results, columns=['Filename', 'Sentence'])

df.to_csv(output_csv, index=False)

print('done')


done


In [3]:
df5=pd.read_csv('/home/c/Downloads/dialogues.csv')
df5.head()
#this may not be so accurate for single quotes the file 'james-eagle-1854_25.txt'

,Filename,Sentence
0,james-eagle-1854_25.txt,'s sister was next in size and importance to\n...
1,james-eagle-1854_25.txt,'s heart were too powerful to leave any\ntimid...
2,james-eagle-1854_25.txt,"""She is come! the Great Spirit has sent her! S..."
3,james-eagle-1854_25.txt,"""Thy child hears thy words,"""
4,james-eagle-1854_25.txt,"""What has the Grey Dove to say to the\ndaughte..."


In [4]:
df_eg=df[df['Filename']=='ainsworth-auriol-1850_2.txt']
df_eg
#this looks OK for this file

,Filename,Sentence
1441409,ainsworth-auriol-1850_2.txt,"'s figure was slight, but admirably proportion..."
1441410,ainsworth-auriol-1850_2.txt,"""What have you done to me?"""
1441411,ainsworth-auriol-1850_2.txt,"""the\ntorture of my wound has suddenly ceased,..."
1441412,ainsworth-auriol-1850_2.txt,"""You are cared for by one who has greater skil..."
1441413,ainsworth-auriol-1850_2.txt,"""If I can manage to transport you to his\nlodg..."
1441414,ainsworth-auriol-1850_2.txt,"""Do not delay, then,"""
1441415,ainsworth-auriol-1850_2.txt,"""for though I am free from\npain, I feel that ..."
1441416,ainsworth-auriol-1850_2.txt,"""Press this handkerchief to your side, and lea..."
1441417,ainsworth-auriol-1850_2.txt,"""Doctor Lamb's dwelling is but a step from the..."
1441418,ainsworth-auriol-1850_2.txt,"""It is the first I ever heard of him,"""


In [5]:
df6=df5.sort_values(by='Filename')


In [6]:
df6.head()

,Filename,Sentence
234269,ainsworth-auriol-1850_1.txt,"""They were father and son,"""
234252,ainsworth-auriol-1850_1.txt,"""THE TOWER OF LONDON"""
234253,ainsworth-auriol-1850_1.txt,'S COPYRIGHT EDITION_\n\n LONDON\n GEORG...
234254,ainsworth-auriol-1850_1.txt,'S ADVENTURE IN ROME--\n\nI. SANTA MARIA MAGGI...
234255,ainsworth-auriol-1850_1.txt,'S DEVICE TO PERPLEX AURIOL\n\n\n\n\nPROLOGUE\...


In [7]:
df6.to_csv("sorted_dialogues.csv", encoding='utf-8', index=False)


In [8]:
df7=pd.read_csv("sorted_dialogues.csv")
df7.head()



,Filename,Sentence
0,ainsworth-auriol-1850_1.txt,"""They were father and son,"""
1,ainsworth-auriol-1850_1.txt,"""THE TOWER OF LONDON"""
2,ainsworth-auriol-1850_1.txt,'S COPYRIGHT EDITION_\n\n LONDON\n GEORG...
3,ainsworth-auriol-1850_1.txt,'S ADVENTURE IN ROME--\n\nI. SANTA MARIA MAGGI...
4,ainsworth-auriol-1850_1.txt,'S DEVICE TO PERPLEX AURIOL\n\n\n\n\nPROLOGUE\...


In [ ]:
#we must check the accuracy of cutting off dialogues. james-eagle-1854_25.txt looks tricky
#There is no way to reach 100% accuracy, though